## PyTorch神经网络基础
### 模型构造

#### 层和块

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)    
)

X = torch.rand(2, 20)
X, net(X)

(tensor([[0.8887, 0.6795, 0.5834, 0.8555, 0.1360, 0.2766, 0.8564, 0.4052, 0.0529,
          0.4533, 0.9186, 0.9740, 0.0550, 0.7916, 0.0833, 0.9865, 0.7495, 0.3609,
          0.1743, 0.2397],
         [0.3476, 0.8417, 0.7710, 0.8932, 0.8506, 0.3515, 0.0022, 0.6177, 0.5104,
          0.3294, 0.0783, 0.2727, 0.2742, 0.4816, 0.8148, 0.5321, 0.6659, 0.7652,
          0.9995, 0.9450]]),
 tensor([[ 0.0298,  0.0818,  0.0727, -0.2150, -0.1671,  0.0273, -0.2102,  0.1119,
           0.1505, -0.1315],
         [-0.1438,  0.1358,  0.2299, -0.2004, -0.1919,  0.0388, -0.2635,  0.3062,
           0.0926, -0.1413]], grad_fn=<AddmmBackward0>))

In [2]:
# 自定义块
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, x):
        return self.out(F.relu(self.hidden(X)))
    

net = MLP()
net(X)

tensor([[ 0.2157,  0.1146,  0.2246, -0.0805, -0.1106,  0.0441,  0.2845,  0.0285,
         -0.0687, -0.0415],
        [ 0.1513,  0.0234,  0.2039, -0.1620,  0.0303, -0.1307,  0.2196, -0.0430,
         -0.0268, -0.1568]], grad_fn=<AddmmBackward0>)

In [3]:
# 顺序块的实现：

class MySequential(nn.Module):
    def __init__(self, *args) -> None:
        super().__init__()
        for block in args:
            self._modules[block] = block
            
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
net(X)

tensor([[-0.2197,  0.0322,  0.0572, -0.1692,  0.0575, -0.0786,  0.1584, -0.1226,
          0.0792, -0.1798],
        [-0.3489, -0.1436, -0.0257, -0.1591, -0.0553, -0.1610,  0.2511, -0.0495,
          0.1098,  0.1603]], grad_fn=<AddmmBackward0>)

In [4]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=True)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0246, grad_fn=<SumBackward0>)

### 参数管理

In [5]:
# 首先关注具有单隐藏层的多层感知机
import torch
from torch import nn

net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.4061],
        [-0.2712]], grad_fn=<AddmmBackward0>)

In [6]:
net, net[2].state_dict(), net[2].bias.data, net[2].weight.grad==None

# 这是一个排好序的dict

(Sequential(
   (0): Linear(in_features=4, out_features=8, bias=True)
   (1): ReLU()
   (2): Linear(in_features=8, out_features=1, bias=True)
 ),
 OrderedDict([('weight',
               tensor([[ 0.1128,  0.1203, -0.1812, -0.2010, -0.0095,  0.3156, -0.1378,  0.3520]])),
              ('bias', tensor([-0.1919]))]),
 tensor([-0.1919]),
 True)

In [7]:
def block1():
    return nn.Sequential(
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU()
    )
    
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.5367],
        [-0.5367]], grad_fn=<AddmmBackward0>)

In [8]:
rgnet

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

#### 内置初始化

In [9]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
        
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0167,  0.0013, -0.0070, -0.0048]), tensor(0.))

In [10]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

#### 对某些块应用不同的初始化方法

In [11]:

def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
net[0].weight.data[0], net[2].weight.data

(tensor([-0.0901, -0.1702,  0.6416, -0.1380]),
 tensor([[42., 42., 42., 42., 42., 42., 42., 42.]]))

In [12]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            'init',
            *[(name, param.shape) for name, param in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
        
net.apply(my_init)
net[0].weight[2]

init weight torch.Size([8, 4])
init weight torch.Size([1, 8])


tensor([0., -0., -0., -0.], grad_fn=<SelectBackward0>)

#### 参数绑定

In [13]:
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    shared,
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8, 1)
)
net(X)

tensor([[-0.3321],
        [-0.3405]], grad_fn=<AddmmBackward0>)

### 自定义层

#### 构造一个没有任何参数的自定义层


In [14]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))
    

tensor([-2., -1.,  0.,  1.,  2.])

#### 将层作为组件合并到更复杂的模型中

In [15]:
net = nn.Sequential(
    nn.Linear(8, 128), 
    CenteredLayer()
)

Y = net(torch.rand(4, 8))
Y.mean()

tensor(-1.3970e-09, grad_fn=<MeanBackward0>)

In [16]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units, ))
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[ 1.3082,  1.9598, -0.9577],
        [ 0.9418,  0.8086,  0.7215],
        [ 0.0882,  0.7332,  1.5645],
        [-0.2416,  0.4162, -0.8241],
        [ 0.3231, -0.8214, -0.0643]], requires_grad=True)

In [17]:
# 使用自定义层直接执行正向传播计算

dense(torch.rand(2, 5))

tensor([[0.1413, 2.9179, 0.0000],
        [0.0000, 1.7878, 0.0000]])

#### 使用自定义层构建模型

In [18]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

### 读写文件

#### 加载和保存张量

In [19]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')


#### 存储一个张量列表，然后将它们读回内存

In [20]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
x2, y2

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

#### 写入或读取从字符串映射到张量的字典

In [21]:
mydict = {
    'x': x,
    'y': y
}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

#### 加载和保存模型参数
##### 将模型的参数存储在一个叫做"---.params"的文件
##### load的时候实例化之后直接读取文件中的参数


In [22]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
X, Y

(tensor([[ 0.0785,  0.3730,  0.8180, -0.8785,  0.1406,  0.8334, -1.2181,  1.1329,
          -1.3274,  2.4472,  0.1494,  1.2209, -0.1896, -1.2449,  0.1998,  0.5163,
          -0.7283, -1.9033,  0.9543, -1.1529],
         [ 0.9938,  1.1632, -0.4163,  0.9026, -0.9093, -0.9339,  0.3191,  0.6374,
           0.0993, -1.6583,  0.2162,  0.7412,  1.7114, -0.4295,  2.1035, -0.0064,
          -0.4811, -1.0811, -2.2240,  1.2712]]),
 tensor([[ 0.1445,  0.0250,  0.2642, -0.1139,  0.5667, -0.2219, -0.0734, -0.2113,
           0.0937, -0.0209],
         [ 0.3766,  0.2342, -0.1945,  0.1804,  0.0216, -0.0368, -0.1745, -0.1769,
           0.1803,  0.0515]], grad_fn=<AddmmBackward0>))

In [23]:
# 使用state_dict将模型的参数存储在文件中
torch.save(net.state_dict(), 'mlp.params')


In [24]:
# 使用load_state_dict进行模型的复制

clone = MLP() # 先实例化
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [25]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])